Validators

In [8]:
def isImplicationCandidate(logic):
    if logic[0] == 'implies' and len(logic) == 3:
        return True
    else:
        return False

In [9]:
def isIFFCandidate(logic):
    if logic[0] == 'iff' and len(logic) == 3:
        return True
    else:
        return False

In [10]:
def isNPCandidate(logic):
    if logic[0] == 'not' and len(logic) == 2 and len(logic[1]) != 1:
        return True
    else:
        return False

In [11]:
def isDistributionCandidate(logic):
    
    if logic[0] == 'or':
        for i in range(1, len(logic)):
            if len(logic[i]) > 1:
                if logic[i][0] == 'and':
                    return True
    return False

Input - The logic is in the form of list.

TODO - Read the txt file and convert it into list.

Step-1: Eliminating the Implication

In [12]:
def eliminateImplication(logic):
    
    result = []
    result.append('or')
    result.append(['not', logic[1]])
    result.append(logic[2])
    
    return result

TODO - Eliminate Equivalence also.

In [13]:
logic = ['if', ['and', 'A', 'B'], 'C']
print(eliminateImplication(logic)) # ['or', ['not', ['and', 'A', 'B']], 'C']

['or', ['not', ['and', 'A', 'B']], 'C']


Step-2: Converting the negation of the implication

∀x,y(P(x,y)) will be written in list form as:

['forall', 'x', ['forall', 'y', 'f(x,y)']] for functions.

['forall', 'x', ['forall', 'y', ['P', 'x', 'y']]] for predicates.

In [14]:
def propagateNOT(logic):
    
    result = []

    #Checking if the logic is a single element
    if type(logic[1]) == str:
        return logic 
    #Checking if the inward logic is OR then append AND
    if(logic[1][0] == 'or'):
        result.append('and')
    #Chicking if inward logic is AND then append OR
    elif(logic[1][0] == 'and'):
        result.append('or')
    #Checking if inward logic is NOT then return the logic alone
    elif(logic[1][0] == 'not'):
        return logic[1][1]
    elif logic[1][0] == 'forall':
        # Create a negated existential quantifier with the same variable and expression
        return ['exists', logic[1][1], propagateNOT(['not', logic[1][2]])]
    # Check if the logic is an existential quantifier (∃) then convert to universal (∀) with NOT
    elif logic[1][0] == 'exists':
        # Create a negated universal quantifier with the same variable and expression
        return ['forall', logic[1][1], propagateNOT(['not', logic[1][2]])]
    else:
        return logic
        
    #For all arguments of the inner list
    for i in range(1, len(logic[1])):
        #check if the first argument is another list
        if len(logic[1][i]) != 1:
            #recursively call to propogate not further inwards
            result.append(propagateNOT(['not', logic[1][i]]))
        else:
            #else append the negation of the single element
            result.append(['not', logic[1][i]])
     
    return result

In [15]:
logic = ['not', ['P', 'x', 'y']]
print(propagateNOT(logic)) # ['not', ['P', 'x', 'y']]

['not', ['P', 'x', 'y']]


In [16]:
logic = ['not', ['and', 'A', 'B']]
print(propagateNOT(logic)) # ['or', ['not', 'A'], ['not', 'B']]

['or', ['not', 'A'], ['not', 'B']]


In [17]:
logic = ['not', ['forall', 'x', 'f(x,y)']]
print(propagateNOT(logic)) # ['exists', 'x', 'A']

['exists', 'x', ['not', 'f(x,y)']]


In [18]:
logic = ['not', ['forall', 'x', ['P', 'x', 'y']]]
print(propagateNOT(logic)) # ['exists', 'x', ['not', ['P', 'x', 'y']]]

['exists', 'x', ['not', ['P', 'x', 'y']]]


Step-3: Eliminating the Existential Quantifier

Note: ∀x(∃y(P(x,y))) after skolemization is P(?x, sk-y(?x)).

TODO: Implement this properly.

In [19]:
def skolemize(fol, universal_vars=[], skolem_counter={'constants': 0, 'functions': 0}):
    """
    Skolemize a FOL formula represented as a list, maintaining list structure.
    
    Args:
    fol: The FOL formula as a list.
    universal_vars: Variables currently universally quantified in the scope.
    skolem_counter: A dictionary with counters for Skolem constants and functions.
    
    Returns:
    The skolemized formula in list form.
    """
    if fol[0] == 'forall':
        # Process the formula within the scope of the universal quantifier, adding the variable to the scope
        return ['forall', fol[1]] + [skolemize(fol[2], universal_vars + [fol[1]], skolem_counter)]
    elif fol[0] == 'exists':
        if len(universal_vars) == 0:
            # Replace with a Skolem constant if no universal variables are in scope
            skc = f'skc{skolem_counter["constants"]}'
            skolem_counter["constants"] += 1
            # Replace the existential variable with the Skolem constant in the predicate
            return replace_variable(fol[2], fol[1], skc)
        else:
            # Replace with a Skolem function if universal variables are in scope
            skf = f'skf{skolem_counter["functions"]}(' + ','.join(universal_vars) + ')'
            skolem_counter["functions"] += 1
            return replace_variable(fol[2], fol[1], skf)
    elif isinstance(fol, list):
        # Recursively process each element of the list if it's a complex expression
        return [skolemize(part, universal_vars, skolem_counter) for part in fol]
    else:
        # Return the element unchanged if it's not a quantifier expression
        return fol

def replace_variable(expr, var, replacement):
    """
    Recursively replace a variable with a Skolem constant/function in an expression.
    
    Args:
    expr: The expression (part of the formula) in which to replace the variable.
    var: The variable to replace.
    replacement: The Skolem constant or function to replace the variable with.
    
    Returns:
    The expression with the variable replaced.
    """
    if isinstance(expr, str):
        # If the expression is a string, replace the variable if it matches
        return expr.replace(var, replacement)
    elif isinstance(expr, list):
        # If the expression is a list, recursively replace in each element
        return [replace_variable(part, var, replacement) for part in expr]
    else:
        # Return the expression unchanged if it's neither a string nor a list
        return expr

# Example FOL formula
fol_formula = ['forall', 'x', ['exists', 'y', ['P', 'x', 'f(y)']]]
skolemized_formula = skolemize(fol_formula)
print(f"Skolemized Formula: {skolemized_formula}")

Skolemized Formula: ['forall', 'x', ['P', 'x', 'f(skf0(x))']]


In [20]:
fol_formula = ['forall', 'x', ['exists', 'y', ['P', 'x', 'y']]]
skolemized_formula = skolemize(fol_formula)
print(f"Skolemized Formula: {skolemized_formula}")

Skolemized Formula: ['forall', 'x', ['P', 'x', 'skf1(x)']]


What to do with universal quantifier?

Step-4: Removing the Universal Quantifier

In [21]:
def remove_universal_quantifiers(fol):
    """
    Remove universal quantifiers ('forall') from a FOL formula.

    Args:
    fol: The FOL formula as a nested list.

    Returns:
    The modified FOL formula as a nested list without 'forall'.
    """
    if not fol or not isinstance(fol, list):
        return fol  # Return as is if fol is not a list or is empty
    
    if fol[0] == 'forall':
        # Skip 'forall' and directly return the processing result of its child without including 'forall' in the output
        return remove_universal_quantifiers(fol[2])
    else:
        # Recursively process each element in the list for nested structures
        return [remove_universal_quantifiers(element) for element in fol]

# Example FOL formula
fol_formula = ['forall', 'x', ['and', ['or', 'A', 'B'], ['forall', 'y', ['P', 'x', 'y']]]]
# Removing universal quantifiers
modified_formula = remove_universal_quantifiers(fol_formula)
print(modified_formula)

['and', ['or', 'A', 'B'], ['P', 'x', 'y']]


Step-5: Distributing union over intersection

In [28]:
def parseDistribution(logic):
      
    #Check if the logic is a candidate for distribution of OR over ANDs
    if isDistributionCandidate(logic):
        logic = distributeOR(logic)
        
    #For all the attributes in the logic repeat the process recursively
    for i in range(1, len(logic)):
        if len(logic[i]) > 1:
            logic[i] = parseDistribution(logic[i])
            
    #Check if the logic is a candidate for distribution of OR over ANDs
    
    if isDistributionCandidate(logic):
        logic = distributeOR(logic)
    
    #return distributed logic
    return simplify_logic(logic)

In [23]:
# def simplify(logic):
    
#     #Only simplify when the literals to an operator are more than 2
#     if len(logic) > 3:
#         #need to simplify
#         logic = getSimplified(logic[0], logic[1:len(logic)-1], logic[len(logic) - 1])
        
#     #Repeat recursively for all sub logics
#     for i in range(1, len(logic)):
#         if len(logic[i]) > 1:
#             logic[i] = simplify(logic[i])
        
#     #Only simplify when the literals to an operator are more than 2
#     if len(logic) > 3:
#         #need to simplify
#         logic = getSimplified(logic[0], logic[1:len(logic)-1], logic[len(logic) - 1])
        
#     return logic

In [24]:
# def getSimplified(operator, literals, current):
    
#     result = []
#     result.append(operator)
    
#     #If the literal length is 1 append the literal else recursively call for simplification
#     if len(literals) == 1:
#         result.append(literals[0])
#     else:
#         result.append(getSimplified(operator, literals[0:len(literals)-1], literals[len(literals)-1]))
        
#     #append the current 
#     result.append(current)
    
#     #return simplified logic
#     return result

In [25]:
def distributeOR(logic):
    
    result = []
    #AND will propogate outwards
    result.append('and')
    
    #Check if both the lists are ands
    if logic[1][0] == 'and' and logic[2][0] == 'and':
        #Distribute the literals 
        result.append(parseDistribution(['or', logic[1][1], logic[2][1]]))
        result.append(parseDistribution(['or', logic[1][1], logic[2][2]]))
        result.append(parseDistribution(['or', logic[1][2], logic[2][1]]))
        result.append(parseDistribution(['or', logic[1][2], logic[2][2]]))
        
    else:
        #Either one is and and
        if logic[1][0] == 'and':
            
            #Check if the second argument is a list
            if len(logic[2]) > 2:
                #check if its a candidate for distribution
                if isDistributionCandidate(logic[2]):
                    logic[2] = parseDistribution(logic[2])
                    
                    #Distribute the literals 
                    result.append(parseDistribution(['or', logic[1][1], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1][1], logic[2][2]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2][2]]))
                
                else:
                    #Keep the second as it is
                    result.append(parseDistribution(['or', logic[1][1], logic[2]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2]]))
                    
            else:
                #Keep the second as it is
                result.append(parseDistribution(['or', logic[1][1], logic[2]]))
                result.append(parseDistribution(['or', logic[1][2], logic[2]]))
        else:
            
            #Check if the second argument is a list
            if len(logic[1]) > 2:
                #check if its a candidate for distribution
                if isDistributionCandidate(logic[1]):
                    logic[1] = parseDistribution(logic[1])
                    
                    #Distribute the literals 
                    result.append(parseDistribution(['or', logic[1][1], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1][1], logic[2][2]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1][2], logic[2][2]]))
                else:
                    #Keep the second as it is
                    result.append(parseDistribution(['or', logic[1], logic[2][1]]))
                    result.append(parseDistribution(['or', logic[1], logic[2][2]]))
            else:
                #Keep the second as it is
                result.append(parseDistribution(['or', logic[1], logic[2][1]]))
                result.append(parseDistribution(['or', logic[1], logic[2][2]]))
            
                
    return simplify(result)

In [26]:
logic = ['or', ['and', 'A', 'B'], ['and', 'C', 'D']]
print(parseDistribution(logic)) # ['and', ['or', 'A', 'C'], ['or', 'A', 'D'], ['or', 'B', 'C'], ['or', 'B', 'D']]

['and', ['and', ['and', ['or', 'A', 'C'], ['or', 'A', 'D']], ['or', 'B', 'C']], ['or', 'B', 'D']]


Condended Simplify and Get_simplified into 1 function:

In [27]:
def simplify_logic(logic):
    """
    Simplify a logical expression to ensure all operators have only two operands.
    
    Args:
    logic: The logical expression to be simplified.
    
    Returns:
    The simplified logical expression.
    """
    # Base case: if logic is not a list or is too short, return it directly.
    if not isinstance(logic, list) or len(logic) <= 2:
        return logic
    
    operator = logic[0]
    operands = logic[1:]
    
    # Recursively simplify all operands first.
    simplified_operands = [simplify_logic(operand) for operand in operands]
    
    # Function to iteratively apply the operator to ensure only binary operations.
    def binary_operation(op, args):
        while len(args) > 2:
            # Repeatedly apply the operator to the last two items and update the list.
            args = args[:-2] + [[op, args[-2], args[-1]]]
        return args

    # Apply binary operation restructuring.
    if len(simplified_operands) > 2:
        # Start the iterative simplification if there are more than two operands.
        simplified_operands = binary_operation(operator, simplified_operands)
    
    # Reconstruct the logic expression with the simplified operands.
    return [operator] + simplified_operands

# Example usage
logic_expression = ['or', 'a', 'b', 'c', 'd']
simplified_expression = simplify_logic(logic_expression)
print(f"Simplified Expression: {simplified_expression}")


Simplified Expression: ['or', 'a', ['or', 'b', ['or', 'c', 'd']]]
